# PROJECT: Wrangle and Clean Data

In [629]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import tweepy as tw
import json
import time

In [619]:
%matplotlib inline

In [620]:
%config InlineBackend.figure_format = "retina"

# Gathering Data

In [621]:
#Loading the tweet archive tweet obtained as a handheld file

archive = pd.read_csv("twitter-archive-enhanced.csv")
archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [622]:
#make a list of twitter ids present in archive table
id_list = list(archive.tweet_id)

## Programmatically downloading image prediction file from url link 
#-- https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv



In [623]:
#make directory --image predictions

folder = "image_prediction"

if not os.path.exists(folder):
    os.mkdir(folder)
    
#Obtain response from url using the request library
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

#load the dcontent of the data into a file in the created folder

with open(os.path.join(folder, url.split("/")[-1]), "wb") as file:
    file.write(response.content)

## Accessing additional twitter information from twitter's API using tweepy

In [624]:
#Accessing twitter developer keys

with open("twitter_keys.txt", "r") as file:
    keys = file.readlines()

API_key = keys[0].strip("\n")
API_key_secret = keys[1].strip("\n")
Bearer_token = keys[2].strip("\n")
Access_token = keys[3].strip("\n")
Access_secret = keys[4]

In [625]:
#initializing the api class using the obtained the deveolper keys

#Authentication
auth = tw.OAuthHandler(API_key, API_key_secret)
auth.set_access_token(Access_token, Access_secret)

api = tw.API(auth, wait_on_rate_limit = True)  #Initializing class

In [626]:
#using api class to access tweet via the tweet id obtained in the first table --archive
start_time = time.time()
count = 0
len_id_list = len(id_list)
error_dict = {}
with open("json-text.txt", "w") as file:
    for tweet_id in id_list:
        count+=1
        print(count, f"out of {len_id_list}")
        try:
            response = api.get_status(tweet_id, tweet_mode = "extended")
            json.dump(response._json, file)
            file.write("\n")
            print("success")
            #continue
        except tw.TweepyException as e:
                print("Failed")
                error_dict[tweet_id] = e
        pass
        pass
    pass
time_taken = time.time() - start_time
print(time_taken)
print(error_dict)

AttributeError: 'str' object has no attribute 'time'

In [628]:
import time
time.time()

1654165844.0826602

In [ ]:
#load each line of the txt json-text.txt file just created and append it
#to empty list json_list
json_list = []
with open("json-text.txt", "r") as file:
    for x in id_list:
        json_list.append(file.readline())

In [ ]:
pd_list = []
len_ = len(json_list)  #obatain length of list to iterate over
for i in len_:
    if not json_list[i]:   #This condition tests if the json_string is empty
        continue
    else:
        json_load = json.loads(json_list[i])
        tweet_id = json_load["id_str"]
        time = json_load["created_at"]
        likes = json_load["favorite_count"]
        retweet_counts = json_load["retweet_count"]
        json_dict = {"tweet_id": tweet_id,
                    "timestamp": time,
                    "likes": likes,
                    "retweet_count": retweet_counts}
    pd_list.append(json_dict)

In [ ]:
additional_info = pd.DataFrame(pd_list)  #use pd.DataFrame for make dataframe

#### Load and print all three tables

In [ ]:
#First table, archives of tweet handed over
archive.head(1)

In [ ]:
#Second tabele stored in image prediction folder

img_prediction = pd.read_csv("./image_prediction/image-predictions.tsv", sep = "\t")
img_prediction.head(2)

In [ ]:
additional_info.head(2)

# Asessing the tables

In [ ]:
archive.to_csv("archive.csv", index = False)
img_prediction.to_csv("img_prediction.csv", index = False)
additional_info.to_csv("add_info.csv", index = False)

In [ ]:
archive.sample()

In [ ]:
archive.info()

From the above we can spot incomplete columns and invalid datatypes as detailedly refreenced below

In order to have a visual view and asssessment of the non null values of the retweet_id columns, we will run the code below

In [ ]:
archive[archive.in_reply_to_status_id.notnull()]

Let us further investigate the unique properties of the dog stages columns and have a clearer of perspective of what data we are dealing with.


In [ ]:
print(archive.pupper.nunique())
print(archive.pupper.unique())

print(archive.puppo.nunique())
print(archive.puppo.unique())

print(archive.doggo.nunique())
print(archive.doggo.unique())

print(archive.floofer.nunique())
print(archive.floofer.unique())

In [ ]:
sum(archive.puppo.isnull()) #This examine to see if empty values are accurately represented

Investigate the source column programmatically to see the column values

In [ ]:
archive.source.unique()

In [ ]:
sum(archive.tweet_id.duplicated()) #Check for duplicates -- found none

In [ ]:
print(archive.rating_numerator.nunique())
print(archive.rating_numerator.unique())

print(archive.rating_denominator.nunique())
print(archive.rating_denominator.unique())

In [ ]:
archive[archive.name == "None"]  #An iterating step brought me back to exmine and check for names recorded as None

**Check for img_prediction table next**

In [ ]:
img_prediction

In [ ]:
img_prediction.info()

In [ ]:
img_prediction.img_num.unique()

In [ ]:
sum(img_prediction.jpg_url.duplicated()) #Check for same image

**Check for additional_info table next**

In [ ]:
additional_info

In [ ]:
additional_info.info()

In [ ]:
additional_info.describe()

In [ ]:
sum(additional_info.tweet_id.duplicated()) #Check for duplicate tweets

### Documenting Assessments from Archive table
##### Quality Issues
- Highly missing values in columns (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp and expanded_urls(few missing values for expanded urls))
- timestamp column contains extra +0000 character at the end and also time values which might not be necessary for analysis
- Wrong datatype
    - tweet_id, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id(string is appropriate)
    - timestamp, retweeted_status_timestamp are not datetime datatype
- doggo, floofer, pupper and puppo columns use None instead of NaN
- Source column contains the soure of each tweet and datatype should be category.
- Some tweets are retweets. Tweets that have non-null retweet_id. They are not regarded as original tweets.
- The rating_numerator and rating_numerator can be combined as one observatory value as rating.
- Recorded name as None

##### Tidyness Issues
- Values as columns in doggo, floofer, pupper and puppo columns


### Documenting Assessments from img_prediction table
##### Quality Issues
- tweet_id should be string
- The first 3 columns seem to be the only relevant for the analysis we want to perform.
- Duplicate image urls
- descriptive column name (img_num)

### Documenting Assessments from additional_info table
##### Quality Issues
- timestamp has extra +0000 characters and should be datetime object

##### Tidyness Issues
- Merge all three tables as tweet-master-archive

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
#make copies of each dataframe

archive_clean = archive.copy()
img_prediction_clean = img_prediction.copy()
additional_info_clean = additional_info.copy()

### Issue #1: 
Archive table - Highly missing values in columns (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp and expanded_urls(few missing values for expanded urls))

The reason for this highly missing values in the column is because of nature of tweet. For example, only tweets that are retweet have retweet_id e.t.c.
    
We are however dealing with original tweets and not retweets

#### Define:

- Slice table for where retweet_id is null and drop all retweet related columns
- Drop the in_reply columns and expanded urls (they contribute little or nothing)

#### Code

In [ ]:
archive_clean = archive_clean[archive_clean.retweeted_status_id.isnull()] #slicing dataframe

In [ ]:
archive_clean.drop(["retweeted_status_id", 
                    "retweeted_status_user_id", "retweeted_status_timestamp"], axis = 1, inplace = True)

In [ ]:
archive_clean.drop(["in_reply_to_status_id", 
                    "in_reply_to_user_id", 
                    "expanded_urls"], axis = 1, inplace = True)

#### Test

Note: The number of non null retweet columns was 181 from a dataframe of originally 2356 entries

In [ ]:
archive_clean.info()

### Issue #2: 
- Values as columns in doggo, floofer, pupper and puppo columns

#### Define:

- Melt the dataframe to have one column as dog_stage and value that shows the actual dog_stage.
- Drop the var_name column
- Sort dataframe by tweet_id, and value_name, then drop dupliactes exploiting the structure of the rearranged df

#### Code

In [ ]:
arch_cols = set(archive_clean.columns)
stage_cols = {"doggo", "floofer", "pupper", "puppo"}
id_vars = list(arch_cols - stage_cols)

In [ ]:
archive_clean = pd.melt(archive_clean, id_vars = id_vars, 
        var_name = "dog_stage_class", 
        value_name="dog_stage")

In [ ]:
#Drop the var_name
archive_clean.drop("dog_stage_class", axis = 1, inplace = True)

#### Test



In [ ]:
archive_clean.columns

In [ ]:
archive_clean.dog_stage.unique()

In [ ]:
sum(archive_clean.dog_stage!="None")

### Issue #3: 
- doggo, floofer, pupper and puppo columns use None instead of NaN

#### Define:

- Replace "None" with np.nan
- Exploit the nature of nan to sort the dataframe by tweet_id and dog_stage, then use this structure to obtain unique tweet ids while retaining values where a particular tweet has a value apart from none.

#### Code

In [ ]:
archive_clean.dog_stage = archive_clean.dog_stage.replace("None", np.nan) #replace with nan

In [ ]:
archive_clean = archive_clean.sort_values(["tweet_id", "dog_stage"], na_position = "last")
archive_clean = archive_clean.drop_duplicates(subset ="tweet_id", keep = "first")

#### Test


In [ ]:
archive_clean.shape  #we are expecting just the shape of the dataset to remain uncanged i.e. 2175 rows

In [ ]:
archive_clean.dog_stage.unique()

In [ ]:
sum(archive_clean.dog_stage.notnull())  
#The value (344 as opposed to 356) here show that there are some rows which posibbly have two different dog stages

### Issue #4: 
- timestamp column contains extra +0000 character at the end and also time values which might not be necessary for analysis.

#### Define:

- slice string to retain valuable characters
- convert column to datetime object

#### Code

In [ ]:
archive_clean.timestamp = archive_clean.timestamp.apply(lambda x: x.split("+")[0]) #string to extract strict date time values

In [ ]:
archive_clean.timestamp = pd.to_datetime(archive_clean.timestamp)  #Conert to datetime object

#### Test

In [ ]:
archive_clean.timestamp.value_counts()

In [ ]:
archive_clean.timestamp.dtype #Check for datatype of the column timestamp

### Issue #5: 
- tweet_id column of archive table datatype to string

#### Define:

- Convert column to string

#### Code

In [ ]:
archive_clean.tweet_id = archive_clean.tweet_id.astype(str)

#### Test

In [ ]:
archive_clean.tweet_id.dtype

### Issue #6: 
- Source column contains the soure of each tweet and datatype should be category.

#### Define:

- Split the column by delimeter ">" and select nth index as appropriate.
- Convert column to category

#### Code

In [ ]:
#split column by delimeter ">" and extract valid string

archive_clean.source = archive_clean.source.apply(lambda x: x.split(">")[1][:-3])

In [ ]:
#convert to category

archive_clean.source = archive_clean.source.astype("category")

#### Test

In [ ]:
archive_clean.source.unique()

### Issue #7: 
- The rating_numerator and rating_numerator can be combined as one observatory value as rating.

#### Define:

- Divide the rating numerator by the denominator and record as rating
- Drop rating_numerator and rating_denominator

#### Code

In [ ]:
archive_clean["rating"] = archive_clean.rating_numerator/archive_clean.rating_denominator

In [ ]:
archive_clean.drop(["rating_numerator", "rating_denominator"], axis = 1, inplace = True)

#### Test

In [ ]:
archive_clean.sample(5)

### Issue #8: 

- The None value of the name column can be left as None or renamed as unnamed or another variable but I would not really support using np.nan as a value, maybe not available because they are tweets that did not reference the name of the particular dog

### Issue #9: 

- img_prediction table: The first 3 columns seem to be the only relevant for the analysis we want to perform

#### Code

In [ ]:
img_prediction_clean = img_prediction_clean[["tweet_id", "jpg_url", "img_num"]]

#### Test

In [ ]:
img_prediction_clean.head(2)

### Issue #10: 

- img_prediction: tweet_id should be string.

#### Code

In [ ]:
img_prediction_clean.tweet_id = img_prediction_clean.tweet_id.astype(str)

#### Test

In [ ]:
img_prediction_clean.tweet_id.dtype

### Issue #11: 

- img_num: Rename img_num to img_classifiacation

#### Code

In [ ]:
img_prediction_clean.rename(columns = {"img_num": "img_class"}, inplace = True)

#### Test

In [ ]:
img_prediction_clean.sample()

### Issue #12: 

- jpg_url duplicate suggests that multiple tweets point to a particular img which is not an uncommon thing. so we will preserve this information and not delete the duplicates

### Issue #13: 

- additional_info: timestamp has extra +0000 characters and should be datetime object

#### Code

In [ ]:
additional_info_clean.timestamp = pd.to_datetime(additional_info_clean.timestamp)

In [ ]:
additional_info_clean.timestamp = pd.to_datetime(additional_info_clean.timestamp.apply(lambda x: str(x).split("+")[0]))
#string to extract strict date time values

#### Test

In [ ]:
additional_info_clean.timestamp.value_counts()

In [ ]:
additional_info_clean.info()

### Tidyness Issues

#### Define
- Merge all three tables to be a master dataframe for classifying and rating dogs

#### Code

In [ ]:
twitter_master_archive = pd.merge(archive_clean, img_prediction_clean, how = "inner", on = "tweet_id")

In order to mege the twiiter master df to the additional information, we will need to drop the timestamp of the additional info

In [ ]:
additional_info_clean.drop("timestamp", axis = 1, inplace = True)

In [ ]:
#merge with twitter master archive

twitter_master_archive = pd.merge(twitter_master_archive, additional_info_clean, how = "inner", on = "tweet_id")

#### Test

In [ ]:
twitter_master_archive.head()

In [ ]:
twitter_master_archive.info()

Convert img_class to categorical variables.

In [ ]:
twitter_master_archive.img_class = twitter_master_archive.img_class.astype("category")

In [ ]:
twitter_master_archive.info()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
#Save master dataframe

twitter_master_archive.to_csv("twitter_archive_master.csv", index = False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

In [ ]:
twitter_master_archive.head()

In [ ]:
twitter_master_archive.describe()

In [ ]:
twitter_master_archive.describe(include = "object")

### Insights:
1. The average rating for each dog is 1.17 approximately which is greater than 1. This tells us about how wedogrates do rate the dogs.

2. The average number of likes/favourites and retweets that they receive per tweet is 7727 and 2250 respectively, while the maximum likes and retweets are 145159 and 70940 respectively

3. We have up to 545 unnamed dogs.

### Visualization

In [ ]:
twitter_master_archive.source.value_counts(normalize = True).plot(kind = "bar")
plt.title("Barplot comapring the count of \ndifferent sources of tweet", fontsize = 14)
plt.ylabel("count", fontsize = 12, weight = "bold")
plt.xlabel("sources of tweet", fontsize = 12, weight = "bold");

Most of the tweets from wedogrates are made from iphone and a very inconsequential amount is made from Twitteer for Web Client, where all others can be overlooked.

In [ ]:
sns.barplot(data = twitter_master_archive, x = "img_class", y = "rating")
plt.title("Barplot of image classification compared to the ratings received")
plt.ylabel("rating", fontsize = 12, weight = "bold")
plt.xlabel("image classification", fontsize = 12, weight = "bold");

- Image class 1 has the highest spread of rating from about 1.0 to 1.4 - This is represented in the wick above each bar in the barplot above.

- The average ratings for each image classification are similar. The image classification 4 however stands to be the highest.

- The average rating is represented by the heights of the bar (without the wicks)

In [ ]:
sns.barplot(data = twitter_master_archive, x = "dog_stage", y = "rating")
plt.title("Barplot of dog stage compared to the ratings received")
plt.ylabel("rating", fontsize = 12, weight = "bold")
plt.xlabel("dog stage", fontsize = 12, weight = "bold");

- The rating pupper is the lowest among other form of dog stage

While rating and favourites count seem to be metrics for evaluating a dog, they both are metrics from different perspective. The likes being the metrics from public opinions (voting), while the rating came from enterprise metrics.

First we want to see how much do ratings and likes count correlate to each other.

In [ ]:
#scatter plot of likes and rating

plt.scatter(twitter_master_archive.rating, twitter_master_archive.likes)
plt.title("Scatter plot of rating against the likes", fontsize = 14)
plt.xlabel("likes count", fontsize = 12, weight = "bold")
plt.ylabel("ratings", fontsize = 12, weight = "bold");

The scatterplot is not properly vizualized because of outliers in the likes count axis. (ratings > 2)

In [ ]:
#Numerical correlation between the metrics of interest
twitter_master_archive.corr()

In [ ]:
#drop index of likes greater than 2
row_to_drop = twitter_master_archive[twitter_master_archive.rating > 2]
ind_to_drop = row_to_drop.index

In [ ]:
#Plot scatterplot of adjusted table
plt.scatter(twitter_master_archive.drop(index = ind_to_drop).rating, twitter_master_archive.drop(index = ind_to_drop).likes)
plt.title("Scatter plot of rating against the likes", fontsize = 14)
plt.xlabel("likes count", fontsize = 12, weight = "bold")
plt.ylabel("ratings", fontsize = 12, weight = "bold");

There is very little correlation as already shown in the correlation table, however the relationship is a positive one.